In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/추출요약_최종본_5분류.csv')

In [ ]:
data.info()

In [ ]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.drop(['Unnamed: 0.1'], axis=1, inplace=True)

In [ ]:
data.head()

# 📌 전체 데이터

In [ ]:
total_text = []
for text in data['번역']:
  total_text.append(text)

In [ ]:
len(total_text)

# 📌 전체 데이터 프레임에 대하여 적용

In [ ]:
import re

for i, text in enumerate(total_text):
  sentences = re.split("[\.?!]\S+", text)

  globals()[f'data{i}'] = []

  for sentence in sentences:
    if (sentence == "" or len(sentence) == 0):
      continue
    temp_dict = dict()
    temp_dict['sentence'] = sentence
    temp_dict['token_list'] = sentence.split()

    globals()[f'data{i}'].append(temp_dict)

  globals()[f'df{i}'] = pd.DataFrame(globals()[f'data{i}'])

# 마지막 데이터 확인
df7069

In [ ]:
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1') 
    R = np.ones(A.shape[0]).reshape(-1,1) 
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1) 

    # iteration
    for _ in range(max_iter): 
        R = df * (A * R) + bias

    return R

In [ ]:
import math
import numpy as np

df_list = [globals()[f'df{i}'] for i in range(7070)]
df_list = [data for data in df_list]

total_summary_list = []

for k, df in enumerate(df_list):

  globals()[f'similarity_matrix{k}'] = []

  for i, row_i in df.iterrows():
    i_row_vec = []
    for j, row_j in df.iterrows():
      if i == j:
        i_row_vec.append(0.0)
      else:
        try:
          intersection = len(set(row_i['token_list']) & set(row_j['token_list']))
          log_i = math.log(len(set(row_i['token_list'])))
          log_j = math.log(len(set(row_j['token_list'])))
          similarity = intersection / (log_i + log_j)
        except:
          pass
        i_row_vec.append(similarity)
    globals()[f'similarity_matrix{k}'].append(i_row_vec)


  weightedGraph = np.array(globals()[f'similarity_matrix{k}'])

  R = pagerank(weightedGraph)

  R = R.sum(axis=1) 
  indexs = R.argsort()[-1:] # 해당 rank 값을 정렬, 값이 높은 1개의 문장 index를 반환

  str_ = []
  for index in sorted(indexs):
      str_.append(df['sentence'][index])
  str_ = " ".join(str_)

  total_summary_list.append(str_)

In [ ]:
print(len(total_summary_list))
print(total_summary_list[0])

In [ ]:
data['추출요약'] = total_summary_list

In [ ]:
data.info()

In [ ]:
data.to_csv('/content/drive/MyDrive/추출요약_재요약.csv')

# 📌 [참고] 단계별 작동 방식 확인

In [ ]:
text = """관리 및 운영
음식물 쓰레기 줄이기
음식 쓰레기를 줄이는 것은 식품 안보의 중요한 측면이다. Queen's의 LeanPath Spark 프로그램은 공공 음식 저울과 디지털 사이니지를 사용하여 우리의 세 개의 주요 식당 중 두 곳의 행동 변화를 촉진하고 적극적으로 음식 낭비를 줄이는 것을 목표로 하고 있습니다. 주방에서는 이 기술을 통해 관리팀과 직원이
폐기물의 영향을 즉시 확인하고 그에 따라 조치를 취합니다. 식당에서는 이 프로그램이 소비자 후 쓰레기를 추적하고 음식 쓰레기를 줄이는 데 도움을 줌으로써 그들이 미칠 수 있는 영향에 대해 디지털 사이니지를 통해 식객들에게 교육한다. Queen's는 지속 가능하고 윤리적인 음식을 선택할 수 있는 Fair Trade 캠퍼스입니다.
우리의 공동체는 지속 가능하고 윤리적인 음식 선택에 접근할 수 있다.
교내에서 환대가 운영되는 모든 곳은 윤리적인 소싱을 우선시하며 공정 거래 옵션을 포함하도록 의무화되어 있습니다. 도널드 고든 센터는 2020년에 공정무역 직장 지정을 받았고, 2021년에는 농업 노동자와 환경을 위한 지속 가능한 관행을 지원하기 위해 노력하면서 공정무역 캠퍼스로 지정되었다."""

In [ ]:
# text를 문장 단위로 나누기

import re
 
text = re.sub(r"\n+", " ", text) # 줄바꿈 기호(한 번 이상 반복) 제거
sentences = re.split("[\.?!]\s+", text) # 구두점을 기준으로 문장 단위로 나누기
sentences

In [ ]:
# 각 문장을 요소 단위로 나누기 (토큰화)

import pandas as pd 

data = []

for sentence in sentences:
  if (sentence == "" or len(sentence) == 0): # 빈 문장인 경우 건너뛰기
    continue
  temp_dict = dict() # 빈 딕셔너리 생성 : 문장과 토큰을 한 딕셔너리로 묶기
  temp_dict['sentence'] = sentence # 문장
  temp_dict['token_list'] = sentence.split() # 토큰 : 띄어쓰기 단위로 토큰화

  data.append(temp_dict)

df = pd.DataFrame(data) # DataFrame에 넣어 깔끔하게 보기

In [ ]:
df

### ✔️ 문장들 간의 유사도 계산

> - 문장 간 유사도 = 두 문장에 동시에 등장하는 요소의 개수 / (log(문장A의 요소 개수) + log(문장B의 요소 개수))
> - sentence similarity = len(intersection) / (log(len(set_A)) + log(len(set_B)))

-> 해당 수식을 사용했을 경우, 최대 값이 1이 아니고 문장의 길이가 길수록 높은 유사도를 갖게 된다. <br>
-> 코사인 유사도(Cosine Similarity)를 유사도 함수로 사용했을 경우, <br>짧은 문장에 민감하게 반응하여 2개의 단어를 가진 문장들이 1개의 단어만 겹쳐도 유사하다고 판단

In [ ]:
# 문장들 간의 유사도 계산

import math

similarity_matrix = [] # 전체 문장들 간의 유사도를 저장할 리스트 선언

for i, row_i in df.iterrows(): 
  i_row_vec = [] # 타깃 문장과 나머지 문장들 간의 유사도를 저장할 리스트 선언
  for j, row_j in df.iterrows():
    if i == j: # 같은 문장인 경우 제외
      i_row_vec.append(0.0)
    else: # 다른 문장인 경우에 한해서 문장 간의 유사도 계산
      intersection = len(set(row_i['token_list']) & set(row_j['token_list'])) # 두 문장에 동시에 등장하는 토큰의 개수
      log_i = math.log(len(set(row_i['token_list']))) # log(문장A의 토큰의 개수)
      log_j = math.log(len(set(row_j['token_list']))) # log(문장B의 토큰의 개수)
      similarity = intersection / (log_i + log_j) # 문장 간 유사도 계산
      i_row_vec.append(similarity) # 두 문장 간 유사도 저장
  similarity_matrix.append(i_row_vec) # 전체 문장들 간의 유사도 저장

In [ ]:
# 문장 유사도 그래프 생성

import numpy as np

weightedGraph = np.array(similarity_matrix)
weightedGraph

In [ ]:
# 문장 유사도 그래프를 통해 문장의 rank를 계산해준다.

# 문장 유사도 그래프에 PageRank를 학습하는 함수 생성

from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30): # x : 문장 유사도 그래프 | df(damping factor) | max_iter : 반복횟수
    assert 0 < df < 1 # 가정 설정문

    # initialize
    A = normalize(x, axis=0, norm='l1') # pagerank 알고리즘의 페이지 중요도를 다 더해서 1로 만들어주는 작업
    R = np.ones(A.shape[0]).reshape(-1,1) # R의 경우, 각 문장의 rank값을 의미하고 맨 처음은 1로 초기화되어 있음 
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1) # bias의 경우, 만족하지 못하고 페이지를 떠나는 확률로 (1 - damping factor)를 의미
    
    # iteration : 각 rank는 weighted graph * rank*damping factor + (1-damping factor)로 계산한다.
    for _ in range(max_iter): 
        R = df * (A * R) + bias

    return R

In [ ]:
# # normalize 예시

# from sklearn.preprocessing import normalize

# test_matrix = [
#     [1,0,1],
#     [1,1,0],
#     [1,1,1]
# ]
# test_matrix = np.array(test_matrix)
# print(normalize(test_matrix, axis=0, norm='l1'))

In [ ]:
# 적용
R = pagerank(weightedGraph)

In [ ]:
print(type(R))

In [ ]:
# R은 각 문장별 다른 문장으로부터 rank를 나눠 받은 상태
R

In [ ]:
# row별로 sum을 하면 각 문장 당 최종 rank 값이 나옴
# rank 높은 순으로 문장을 뽑아 나열하면 추출요약이 됨

R = pagerank(weightedGraph) # pagerank를 돌려서 rank matrix 반환
R = R.sum(axis=1) # 반환된 matrix를 row 별로 sum = 최종 rank 값이 나옴
indexs = R.argsort()[-3:] # 해당 rank 값을 정렬, 값이 높은 3개의 문장 index를 반환
print(indexs)
print(sorted(indexs))

# rank값이 높은 문장을 프린트
for index in sorted(indexs): # index를 sorted 하는 이유 : 원래 문장 순서에 맞춰 보여주기 위함
    print(df['sentence'][index])

# 📌 [추가] 문장 요소를 띄어쓰기가 아닌 형태소 분석기로 나누기

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt, Komoran, Kkma, Hannanum, Mecab

okt = Okt()
komoran = Komoran()
kkma = Kkma()
hannanum = Hannanum()
mecab = Mecab()

token_data_okt = []
token_data_komoran = []
token_data_kkma = []
token_data_hannanum = []
token_data_mecab = []

for i, row in df.iterrows():
  sentence = row['sentence']
  # okt
  token_list_okt = komoran.nouns(sentence)
  token_data_okt.append(token_list_okt)
  # komoran
  token_list_komoran = komoran.nouns(sentence)
  token_data_komoran.append(token_list_komoran)
  # kkma
  token_list_kkma = komoran.nouns(sentence)
  token_data_kkma.append(token_list_kkma)
  # hannanum
  token_list_hannanum = komoran.nouns(sentence)
  token_data_hannanum.append(token_list_hannanum)
  # mecab
  token_list_mecab = komoran.nouns(sentence)
  token_data_mecab.append(token_list_mecab)

In [ ]:
df['token_okt'] = token_data_okt
df['token_komoran'] = token_data_komoran
df['token_kkma'] = token_data_kkma
df['token_hannanum'] = token_data_hannanum
df['token_mecab'] = token_data_mecab

In [ ]:
df

# 이후 문장들 간 유사도 계산 부분 반복

# 📌 [추가] 문장추출요약 with Gensim_summarization

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
text_hannanum = """관리 및 운영
음식물 쓰레기 줄이기
음식 쓰레기를 줄이는 것은 식품 안보의 중요한 측면이다
Queen's의 LeanPath Spark 프로그램은 공공 음식 저울과 디지털 사이니지를 사용하여 우리의 세 개의 주요 식당 중 두 곳의 행동 변화를 촉진하고 적극적으로 음식 낭비를 줄이는 것을 목표로 하고 있습니다
주방에서는 이 기술을 통해 관리팀과 직원이
폐기물의 영향을 즉시 확인하고 그에 따라 조치를 취합니다
식당에서는 이 프로그램이 소비자 후 쓰레기를 추적하고 음식 쓰레기를 줄이는 데 도움을 줌으로써 그들이 미칠 수 있는 영향에 대해 디지털 사이니지를 통해 식객들에게 교육한다
Queen's는 지속 가능하고 윤리적인 음식을 선택할 수 있는 Fair Trade 캠퍼스입니다
우리의 공동체는 지속 가능하고 윤리적인 음식 선택에 접근할 수 있다
교내에서 환대가 운영되는 모든 곳은 윤리적인 소싱을 우선시하며 공정 거래 옵션을 포함하도록 의무화되어 있습니다
도널드 고든 센터는 2020년에 공정무역 직장 지정을 받았고, 2021년에는 농업 노동자와 환경을 위한 지속 가능한 관행을 지원하기 위해 노력하면서 공정무역 캠퍼스로 지정되었다."""

In [ ]:
print(summarize(text_hannanum, ratio=0.1))